In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
scope = sy.Define_Scope()

raw_data = sy.Get_Data_All_Parallel(scope)

dt_s=dt(2022,5,1)
dt_e=dt(2022,5,6)

freq_start='1D'
freq_end='1D'

ref_year_s=dt(2022,11,4)

In [6]:
raw_data['multi_ww_df']=um.generate_weather_windows_df(raw_data['w_w_df_all']['hist'], date_start=dt_s, date_end=dt_e, ref_year_start=ref_year_s, freq_start=freq_start, freq_end=freq_end)

100%|██████████| 6/6 [00:00<00:00, 150.39it/s]


In [7]:
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'blooming_progress', 'w_df_all', 'w_w_df_all', 'multi_ww_df'])

In [8]:
model_df = pd.concat([raw_data['yield'],raw_data['multi_ww_df']], sort=True, axis=1, join='inner')
model_df=model_df.dropna()
model_df

,Value,USA_Sdd30_May01-May01,USA_TempMax_May01-May01,USA_Prec_May01-May01,USA_Sdd30_May01-May02,USA_TempMax_May01-May02,USA_Prec_May01-May02,USA_Sdd30_May01-May03,USA_TempMax_May01-May03,USA_Prec_May01-May03,...,USA_Prec_May04-May06,USA_Sdd30_May05-May05,USA_TempMax_May05-May05,USA_Prec_May05-May05,USA_Sdd30_May05-May06,USA_TempMax_May05-May06,USA_Prec_May05-May06,USA_Sdd30_May06-May06,USA_TempMax_May06-May06,USA_Prec_May06-May06
year,,,,,,,,,,,,,,,,,,,,,
1985,34.1,0.000000,23.115531,7.740059,0.000000,23.115531,7.740059,0.000000,23.115531,7.740059,...,0.172154,0.000000,24.673521,0.451683,0.000000,24.673521,0.451683,0.000000,23.908775,3.123178
1986,33.3,0.000000,21.501045,8.649489,0.000000,21.501045,8.649489,0.000000,21.501045,8.649489,...,0.171409,0.000000,25.347209,0.152654,0.000000,25.347209,0.152654,0.000000,26.536711,0.870573
1987,33.9,0.000000,21.297842,0.432377,0.000000,21.297842,0.432377,0.000000,21.297842,0.432377,...,4.264882,0.000000,19.216185,1.097626,0.000000,19.216185,1.097626,0.000000,21.071534,0.564795
1988,27.0,0.000000,23.704131,0.106059,0.000000,23.704131,0.106059,0.000000,23.704131,0.106059,...,1.723140,0.000000,21.203009,0.567803,0.000000,21.203009,0.567803,0.000000,24.053469,0.420053
1989,32.3,0.000000,14.632033,1.745906,0.000000,14.632033,1.745906,0.000000,14.632033,1.745906,...,1.818842,0.000000,17.432464,5.319978,0.000000,17.432464,5.319978,0.000000,13.640826,0.907715
1990,34.1,0.000000,15.814440,1.148494,0.000000,15.814440,1.148494,0.000000,15.814440,1.148494,...,12.984260,0.000000,17.747279,7.538491,0.000000,17.747279,7.538491,0.000000,18.500884,2.127709
1991,34.2,0.000000,17.491689,2.030607,0.000000,17.491689,2.030607,0.000000,17.491689,2.030607,...,7.730341,0.000000,16.436025,12.122465,0.000000,16.436025,12.122465,0.000000,15.260038,6.032193
1992,37.6,0.470846,26.922572,0.087647,0.470846,26.922572,0.087647,0.470846,26.922572,0.087647,...,0.150039,0.000000,18.133409,0.660497,0.000000,18.133409,0.660497,0.000000,15.633441,0.172291
1993,32.6,0.000000,19.373039,2.618089,0.000000,19.373039,2.618089,0.000000,19.373039,2.618089,...,6.674181,0.000000,21.082222,3.537368,0.000000,21.082222,3.537368,0.000000,24.661298,1.114149
